In [33]:
from pennylane import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pennylane as qml
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

Data Dimentionality Reduction

In [37]:
def powerFaultData(method = 'pca'):

	if method == 'pca':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/pca3.csv')
		
		X = []
		Y = []

		X = powerData[['pc1', 'pc2', 'pc3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		X, xt, Y, yt = train_test_split(X, Y, test_size = 0.985, shuffle=True, random_state=42)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.20, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain, xTest, yTest
	
	if method == 'fda':
		powerData = pd.read_csv('/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/Code/DR_data/lda3.csv')
		
		X = []
		Y = []

		X = powerData[['LD1', 'LD2', 'LD3']]
		Y = powerData[['fault']]
		Y = np.asarray(Y).astype(int)

		xTrain, xTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.95, shuffle=True, random_state=42)
		xTrain = preprocessing.normalize(xTrain)
		xTest = preprocessing.normalize(xTest)

		return xTrain, yTrain, xTest, yTest

X, Y, X_test, Y_test = powerFaultData('pca')
Y = Y.ravel()
print("Data: ", X[0])
print("Label: ", Y[0])

Kernel Alignment with Variation Circuit

In [40]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [63]:
dev = qml.device("default.qubit", wires=3, shots=None)
wires = dev.wires.tolist()

def layer(x, params, wires, i0=0, inc=1):
    
    i = i0
    for j, wire in enumerate(wires):
        qml.Hadamard(wires=[wire])
        qml.RZ(x[i % len(x)] * params[2, 0], wires=[wire])
        #qml.RZ(x[i], wires=[wire])
        i += inc
        qml.RY(params[0, j], wires=[wire])

    qml.broadcast(unitary=qml.CRZ, pattern="ring", wires=wires, parameters=params[1])


def ansatz(x, params, wires):
    """The embedding ansatz"""
    for j, layer_params in enumerate(params):
        layer(x, layer_params, wires, i0=j * len(wires))
        

adjoint_ansatz = qml.adjoint(ansatz)

def random_params(num_wires, num_layers):
    """Generate random variational parameters in the shape for the ansatz."""
    params = np.random.uniform(0, 2 * np.pi, (1, num_layers * num_wires), requires_grad=True)
    return np.random.uniform(0, 2 * np.pi, (num_layers, 3, num_wires), requires_grad=True)

@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [42]:
init_params = random_params(num_wires= 3, num_layers = 6)
len(init_params[0])

3

In [43]:
kernel_value = kernel(X[0], X[1], init_params)
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")
print('---------------------------------------------------------------------------------------------')
print('Quantum Circuit: ')
drawer = qml.draw(kernel_circuit)
print(drawer(X[0], X[1], init_params))
print('---------------------------------------------------------------------------------------------')

The kernel value between the first and second datapoint is 0.356
---------------------------------------------------------------------------------------------
Quantum Circuit: 
0: ──H──RZ(1.36)──RY(4.58)─╭●──────────────────╭RZ(2.01)──H──RZ(0.34)──RY(5.25)─╭●─────────────────
1: ──H──RZ(2.40)──RY(3.08)─╰RZ(4.12)─╭●────────│──────────H──RZ(0.60)──RY(1.06)─╰RZ(4.29)─╭●───────
2: ──H──RZ(0.37)──RY(3.15)───────────╰RZ(3.46)─╰●─────────H──RZ(0.09)──RY(4.05)───────────╰RZ(3.03)

──╭RZ(2.37)──H──RZ(1.43)──RY(4.07)─╭●──────────────────╭RZ(1.63)──H──RZ(1.60)──RY(0.17)─╭●───────
──│──────────H──RZ(2.51)──RY(2.64)─╰RZ(4.26)─╭●────────│──────────H──RZ(2.82)──RY(2.27)─╰RZ(3.08)
──╰●─────────H──RZ(0.39)──RY(3.58)───────────╰RZ(3.23)─╰●─────────H──RZ(0.43)──RY(6.09)──────────

────────────╭RZ(1.12)──H──RZ(0.52)──RY(6.20)─╭●──────────────────╭RZ(3.90)──H──RZ(2.35)──RY(3.32)
──╭●────────│──────────H──RZ(0.91)──RY(1.62)─╰RZ(3.27)─╭●────────│──────────H──RZ(4.14)──RY(0.48)
──╰RZ(3.53)─╰●─────────H──RZ(0.

In [44]:
trained_kernel = lambda x1, x2: kernel(x1, x2, init_params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)
accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

The accuracy of a kernel with trained parameters is 0.763


In [45]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_trained.predict(X_test)
#accuracy = accuracy_score(Y_test, y_pred)
report = classification_report(Y_test, y_pred)

/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and bein

In [48]:
accuracy

0.875

In [ ]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [52]:
print("Kernel Alignment with Gradient Descent")
params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(50):
    subset = np.random.choice(list(range(len(X))), 4)
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
                                                X,
                                                Y,
                                                lambda x1, x2: kernel(x1, x2, params),
                                                assume_normalized_kernel=True,
                                            )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_aligned = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_aligned, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent
Step 10 - Alignment = 0.635
Step 20 - Alignment = 0.649
Step 30 - Alignment = 0.667
Step 40 - Alignment = 0.672
Step 50 - Alignment = 0.687
The accuracy of a kernel with trained parameters is 0.774


In [53]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_aligned.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
report = classification_report(Y_test, y_pred)
print(accuracy)
print(report)

0.7916666666666666
              precision    recall  f1-score   support

           1       0.75      0.75      0.75         4
           2       0.60      0.75      0.67         4
           3       1.00      1.00      1.00         2
           4       0.00      0.00      0.00         1
           5       0.60      1.00      0.75         3
           6       1.00      0.80      0.89        10

    accuracy                           0.79        24
   macro avg       0.66      0.72      0.68        24
weighted avg       0.80      0.79      0.78        24



/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and bein

In [64]:
def uncertinitySamplingSubset(X, svm_trained, subSize, sampling = 'entropy', ranking = False):
	
	if sampling == 'entropy':
		if ranking:
			probabilities = svm_trained.predict_proba(X)
			entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
		
			sorted_indices = np.argsort(entropy)
			sorted_entropy_values = np.sort(entropy)

			probabilities = np.linspace(1, 0, len(entropy))
			probabilities = probabilities / probabilities.sum()

			sampled_indices = np.random.choice(sorted_indices, size=subSize, p=probabilities)

			return sampled_indices
		
		else:
			probabilities = svm_trained.predict_proba(X)
			entropy = -np.sum(probabilities * np.log(probabilities), axis=1)
			selected_indices = np.argsort(entropy)[:subSize]

			return selected_indices
	
	return None

def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [65]:
print("Kernel Alignment with Gradient Descent with Active Learning")
"""

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix, probability=True)
svm_trained.fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")
"""
params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(50):
    # Choose subset of datapoints to compute the KTA on.
    
    #subset = np.random.choice(list(range(len(X))), 4) 
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
    svm_trained = SVC(kernel=trained_kernel_matrix, probability=True).fit(X, Y)
    
    subset = uncertinitySamplingSubset(X, svm_trained=svm_trained, subSize=4)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent with Active Learning


KeyboardInterrupt: 

In [58]:
print("Kernel Alignment with Gradient Descent with Active Learning")
"""

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix, probability=True)
svm_trained.fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")
"""
params = init_params
opt = qml.GradientDescentOptimizer(0.2)

for i in range(50):
    # Choose subset of datapoints to compute the KTA on.
    
    #subset = np.random.choice(list(range(len(X))), 4) 
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
    svm_trained = SVC(kernel=trained_kernel_matrix, probability=True).fit(X, Y)
    
    subset = uncertinitySamplingSubset(X, svm_trained=svm_trained, subSize=4, ranking=True)

    #print(type(subset))
    #print(type(Y))
    
    # Define the cost function for optimization
    cost = lambda _params: -target_alignment(
        X[subset],
        Y[subset],
        lambda x1, x2: kernel(x1, x2, _params),
        assume_normalized_kernel=True,
    )
    
    # Optimization step
    params = opt.step(cost, params)

    # Report the alignment on the full dataset every 50 steps.
    if (i + 1) % 10 == 0:
        current_alignment = target_alignment(
            X,
            Y,
            lambda x1, x2: kernel(x1, x2, params),
            assume_normalized_kernel=True,
        )
        print(f"Step {i+1} - Alignment = {current_alignment:.3f}")

trained_kernel = lambda x1, x2: kernel(x1, x2, params)
trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)
svm_trained = SVC(kernel=trained_kernel_matrix).fit(X, Y)

accuracy_trained = accuracy(svm_trained, X, Y)
print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Kernel Alignment with Gradient Descent with Active Learning
Step 10 - Alignment = 0.647
Step 20 - Alignment = 0.674
Step 30 - Alignment = 0.686
Step 40 - Alignment = 0.691
Step 50 - Alignment = 0.716
The accuracy of a kernel with trained parameters is 0.731


In [60]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = svm_trained.predict(X_test)
accuracy = accuracy_score(Y_test, y_pred)
report = classification_report(Y_test, y_pred)
print(accuracy)
print(report)

0.7916666666666666
              precision    recall  f1-score   support

           1       0.75      0.75      0.75         4
           2       0.60      0.75      0.67         4
           3       1.00      1.00      1.00         2
           4       0.00      0.00      0.00         1
           5       0.60      1.00      0.75         3
           6       1.00      0.80      0.89        10

    accuracy                           0.79        24
   macro avg       0.66      0.72      0.68        24
weighted avg       0.80      0.79      0.78        24



/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/digvijay/Developer/MasterThesis/master-thesis-quantumkernelclassification/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and bein